In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_edq_by_store_sku_current AS
WITH
edq_by_store AS (
	SELECT
		store_id,
		product_id,
		edq,
		reason
	FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.winare_edq_by_store
	WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN from_date AND to_date
),
edq_by_city AS (
	SELECT
		b.store_id,
		a.product_id,
		a.edq,
		a.reason
	FROM
		{catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.winare_edq_by_city a
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b USING(city)
	WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN a.from_date AND a.to_date
),
edq_by_mien AS (
	SELECT
		b.store_id,
		a.product_id,
		a.edq,
		a.reason
	FROM
		{catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.winare_edq_by_mien a
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b USING(mien)
	WHERE DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) BETWEEN a.from_date AND a.to_date
),
store_sku AS (
	SELECT DISTINCT
		store_id,
		product_id
	FROM (
		SELECT store_id, product_id FROM edq_by_store
		UNION ALL
		SELECT store_id, product_id FROM edq_by_city
		UNION ALL
		SELECT store_id, product_id FROM edq_by_mien
	)
)
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 calday,
    store_id,
    product_id,
    IFNULL(b.edq, IFNULL(c.edq, d.edq)) edq,
    IF(b.edq IS NOT NULL, 'EDQ BY STORE', IF(c.edq IS NOT NULL, 'EDQ BY CITY',IF(d.edq IS NOT NULL, 'EDQ BY MIEN', 'NOT FOUND'))) note,
	IF(b.edq IS NOT NULL, b.reason, IF(c.edq IS NOT NULL, c.reason, IF(d.edq IS NOT NULL, d.reason, 'NOT FOUND'))) reason
FROM
    store_sku a
    LEFT JOIN edq_by_store b USING(store_id,product_id)
    LEFT JOIN edq_by_city c USING(store_id,product_id)
    LEFT JOIN edq_by_mien d USING(store_id,product_id)
''')